In [1]:
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
import gc


class RNN_model(torch.nn.Module):
    """
    pytorch implementation of neural network
    """
    def __init__(self, input_dim, output_dim, hidden_dim, n_layers):
        """
        :param hidden_dim: int, hidden dimension of lstm (usually 300)
        :param input_dim: int, input dimension of the acoustic features for 1 frame mfcc (usually 429)
        :param output_dim: int, # of trajectories to predict (usually 18)
        :param batch_size:  int, usually 10
        """
        super(Model, self).__init__()
        
        # dimensions 
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        
        
        # LSTM Layer
        self.lstm_layer = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim, num_layers=1,
                                        bidirectional=True)
        self.lstm_layer_2= torch.nn.LSTM(input_size=hidden_dim*2,
                                       hidden_size=hidden_dim, num_layers=1,
                                      bidirectional=True)
       

        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=output_dim)
        self.tanh = torch.nn.Tanh()
        
        self.all_training_loss = []
        self.all_validation_loss = []
        self.all_test_loss = []

        

SyntaxError: invalid syntax (3282599233.py, line 1)

In [3]:
https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb
https://www.simplilearn.com/tutorials/deep-learning-tutorial/rnn
https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
https://slazebni.cs.illinois.edu/spring17/lec26_audio.pdf
    
https://towardsdatascience.com/audio-deep-learning-made-simple-automatic-speech-recognition-asr-how-it-works-716cfce4c706

SyntaxError: invalid syntax (2015319602.py, line 1)